In [1]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

import spynnaker8 as pynn

import numpy as np
import matplotlib.pyplot as plt
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility.plotting import Figure, Panel

simulator_Name = 'spiNNaker'
# exec('import pyNN.%s as pynn' % simulator_Name)


def poisson_generator(rate, rng, t_start=0.0, t_stop=1000.0, array=True,
                      debug=False):
    """
    Returns a SpikeTrain whose spikes are a realization of a Poisson process
    with the given rate (Hz) and stopping time t_stop (milliseconds).

    Note: t_start is always 0.0, thus all realizations are as if
    they spiked at t=0.0, though this spike is not included in the SpikeList.

    Inputs:
        rate    - the rate of the discharge (in Hz)
        t_start - the beginning of the SpikeTrain (in ms)
        t_stop  - the end of the SpikeTrain (in ms)
        array   - if True, a numpy array of sorted spikes is returned,
                  rather than a SpikeTrain object.

    Examples:
        >> gen.poisson_generator(50, 0, 1000)
        >> gen.poisson_generator(20, 5000, 10000, array=True)

    See also:
        inh_poisson_generator, inh_gamma_generator,
        inh_adaptingmarkov_generator
    """

    n = (t_stop - t_start) / 1000.0 * rate
    number = np.ceil(n + 3 * np.sqrt(n))
    if number < 100:
        number = min(5 + np.ceil(2 * n), 100)

    if number > 0:
        isi = rng.exponential(1.0 / rate, number) * 1000.0
        if number > 1:
            spikes = np.add.accumulate(isi)
        else:
            spikes = isi
    else:
        spikes = np.array([])

    spikes += t_start
    i = np.searchsorted(spikes, t_stop)

    extra_spikes = []
    if i == len(spikes):
        # ISI buf overrun

        t_last = spikes[-1] + rng.exponential(1.0 / rate, 1)[0] * 1000.0

        while (t_last < t_stop):
            extra_spikes.append(t_last)
            t_last += rng.exponential(1.0 / rate, 1)[0] * 1000.0

        spikes = np.concatenate((spikes, extra_spikes))

        if debug:
            print("ISI buf overrun handled. len(spikes)=%d,"
                  " len(extra_spikes)=%d" % (len(spikes), len(extra_spikes)))

    else:
        spikes = np.resize(spikes, (i,))

    if debug:
        return spikes, extra_spikes
    else:
        return [round(x) for x in spikes]


# Total number of neurons
Neurons = 3000
sim_time = 1000.0
g = 5.0
eta = 2.0
delay = 2.0
epsilon = 0.1

tau_m = 20.0  # ms (20ms will give a FR of 20hz)
tau_ref = 2.0
v_reset = 10.0
V_th = 20.0
v_rest = 0.0
tauSyn = 1.0

N_E = int(round(Neurons * 0.8))
N_I = int(round(Neurons * 0.2))

C_E = N_E * 0.1
C_I = N_I * 0.1

# Excitatory and inhibitory weights
J_E = 0.1
J_I = -g * J_E

# The firing rate of a neuron in the external pop
# is the product of eta time the threshold rate
# the steady state firing rate which is
# needed to bring a neuron to threshold.
nu_ex = eta * V_th / (J_E * C_E * tau_m)

# population rate of the whole external population.
# With CE neurons the pop rate is simply the product
# nu_ex*C_E  the factor 1000.0 changes the units from
# spikes per ms to spikes per second.
p_rate = 1000.0 * nu_ex * C_E
print("Rate is: %f HZ" % (p_rate / 1000))

# Neural Parameters
pynn.setup(timestep=1.0, min_delay=1.0, max_delay=16.0)

if simulator_Name == "spiNNaker":

    # Makes it easy to scale up the number of cores
    pynn.set_number_of_neurons_per_core(pynn.IF_curr_exp, 100)
    pynn.set_number_of_neurons_per_core(pynn.SpikeSourcePoisson, 100)

_rng = NumpyRNG(seed=1)

v_distr_exc = RandomDistribution('uniform', low=-10.0, high=0.0, rng=_rng)
v_distr_inh = RandomDistribution('uniform', low=-10.0, high=0.0, rng=_rng)

exc_cell_params = {
    'cm': 1.0,  # pf
    'tau_m': tau_m,
    'tau_refrac': tau_ref,
    'v_rest': v_rest,
    'v_reset': v_reset,
    'v_thresh': V_th,
    'tau_syn_E': tauSyn,
    'tau_syn_I': tauSyn,
    'i_offset': 0.9
}

inh_cell_params = {
    'cm': 1.0,  # pf
    'tau_m': tau_m,
    'tau_refrac': tau_ref,
    'v_rest': v_rest,
    'v_reset': v_reset,
    'v_thresh': V_th,
    'tau_syn_E': tauSyn,
    'tau_syn_I': tauSyn,
    'i_offset': 0.9
}

# Set-up pynn Populations
E_pop = pynn.Population(
    N_E, pynn.IF_curr_exp(**exc_cell_params), label="E_pop")

I_pop = pynn.Population(
    N_I, pynn.IF_curr_exp(**inh_cell_params), label="I_pop")

Poiss_ext_E = pynn.Population(
    N_E, pynn.SpikeSourcePoisson(rate=10.0), label="Poisson_pop_E")
Poiss_ext_I = pynn.Population(
    N_I, pynn.SpikeSourcePoisson(rate=10.0), label="Poisson_pop_I")

# Connectors
E_conn = pynn.FixedProbabilityConnector(epsilon)
I_conn = pynn.FixedProbabilityConnector(epsilon)

# Use random delays for the external noise and
# set the inital membrance voltage below the resting potential
# to avoid the overshoot of activity in the beginning of the simulation
_rng = NumpyRNG(seed=1)
delay_distr = RandomDistribution('uniform', low=1.0, high=16.0, rng=_rng)
Ext_conn = pynn.OneToOneConnector()

uniformDistr = RandomDistribution('uniform', low=-10, high=0, rng=_rng)
E_pop.initialize(v=uniformDistr)
I_pop.initialize(v=uniformDistr)

# Projections
E_E = pynn.Projection(
    E_pop, E_pop, E_conn, receptor_type="excitatory",
    synapse_type=pynn.StaticSynapse(weight=J_E, delay=delay))
I_E = pynn.Projection(
    I_pop, E_pop, I_conn, receptor_type="inhibitory",
    synapse_type=pynn.StaticSynapse(weight=J_I, delay=delay))
E_I = pynn.Projection(
    E_pop, I_pop, E_conn, receptor_type="excitatory",
    synapse_type=pynn.StaticSynapse(weight=J_E, delay=delay))
I_I = pynn.Projection(
    I_pop, I_pop, I_conn, receptor_type="inhibitory",
    synapse_type=pynn.StaticSynapse(weight=J_I, delay=delay))

Ext_E = pynn.Projection(
    Poiss_ext_E, E_pop, Ext_conn, receptor_type="excitatory",
    synapse_type=pynn.StaticSynapse(weight=J_E * 10, delay=delay_distr))
Ext_I = pynn.Projection(
    Poiss_ext_I, I_pop, Ext_conn, receptor_type="excitatory",
    synapse_type=pynn.StaticSynapse(weight=J_E * 10, delay=delay_distr))

# Record stuff
E_pop.record("spikes")

pynn.run(sim_time)

esp = None
isp = None
pe = None
pi = None
v_esp = None

esp = E_pop.get_data("spikes")


Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(esp.segments[0].spiketrains,
          yticks=True, markersize=1, xlim=(0, sim_time)),
    title="Brunnel example",
    annotations="Simulated with {}".format(pynn.name())
)
plt.show()

pynn.end()


2022-01-02 23:45:40 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2022-01-02 23:45:40 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2022-01-02 23:45:40 INFO: Setting time scale factor to 1.
2022-01-02 23:45:40 INFO: Setting machine time step to 1000 micro-seconds.
2022-01-02 23:45:40 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us
2022-01-02 23:45:40 INFO: Starting execution process


Detected PyNN version 0.9.4 and Neo version 0.6.1
Rate is: 20.000000 HZ
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2022-01-02 23:45:43 INFO: Time 0:00:03.066471 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2022-01-02 23:45:52 INFO: Time 0:00:08.548766 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2022-01-02 23:45:57 INFO: Time 0:00:04.990577 taken by PartitionAndPlacePartitioner
Created spalloc job 6226471
2022-01-02 23:45:57 INFO: Created spalloc job 6226471
Waiting for board power commands to complete.
2022-01-02 23:45:57 INFO: Waiting for board power commands to complete.
2022-01-02 23:46:02 INFO: Time 0:00:05.048578 taken by SpallocAllocator
2022-01-02 23:46:02 INFO: Creating transceiver for 10.11.193.81
2022-01-02 23:46:02 INFO: Working out if machine is booted
2022-01-02 23:46:06 INFO: Attempting to

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …